# 1 气象数据处理
## CSV文件表头
csv文件结构如下
```
## Site Characteristics
Country     = 'Netherlands'
Station     = 'Wageningen, Haarweg'
Description = 'Observed data from Station Haarweg in Wageningen'
Source      = 'Meteorology and Air Quality Group, Wageningen University'
Contact     = 'Peter Uithol'
Longitude = 5.67; Latitude = 51.97; Elevation = 7; AngstromA = 0.18; AngstromB = 0.55; HasSunshine = False
## Daily weather observations (missing values are NaN)
DAY,IRRAD,TMIN,TMAX,VAP,WIND,RAIN,SNOWDEPTH
20040101,NaN,-0.7,1.1,0.55,3.6,0.5,NaN
20040102,3888,-7.5,0.9,0.44,3.1,0,NaN
20040103,2074,-6.8,-0.5,0.45,1.8,0,NaN
20040104,1814,-3.6,5.9,0.66,3.2,2.5,NaN
20040105,1469,3,5.7,0.78,2.3,1.3,NaN
[...]

with
IRRAD in kJ/m2/day or hours
TMIN and TMAX in Celsius (°C)
VAP in kPa
WIND in m/sec
RAIN in mm
SNOWDEPTH in cm
```


In [1]:
import copy
import yaml
import numpy as np

In [2]:
## 两日相隔天数
from datetime import datetime
def calDays(start, end, format="%Y-%m-%d"):
    strptime, strftime = datetime.strptime, datetime.strftime
    days = (strptime(end, format) - strptime(start, format)).days
    return days

In [3]:
def csv_header(country="China", station="Yangling", longitude=108.12, latitude=34.33, elevation=521):
    res_s = f"## Site Characteristics\nCountry     = {country}\nStation     = {station}\nDescription = 'Observed data from Station {station} in {country}'\
    \nSource      = 'Meteorology and Air Quality Group, Wageningen University'\nContact     = 'Peter Uithol'\
    \nLongitude = {longitude:.2f}; Latitude = {latitude:.2f}; Elevation = {elevation:.0f}; AngstromA = 0.18; AngstromB = 0.55; HasSunshine = False\
    \n## Daily weather observations (missing values are NaN)"
    return res_s

In [4]:
print(csv_header())

## Site Characteristics
Country     = China
Station     = Yangling
Description = 'Observed data from Station Yangling in China'    
Source      = 'Meteorology and Air Quality Group, Wageningen University'
Contact     = 'Peter Uithol'    
Longitude = 108.12; Latitude = 34.33; Elevation = 521; AngstromA = 0.18; AngstromB = 0.55; HasSunshine = False    
## Daily weather observations (missing values are NaN)


# 2 读取模型参数

In [5]:
%matplotlib inline
import sys, os
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
import pandas as pd

data_dir = os.path.join(os.getcwd(), "data")
import pcse
print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

This notebook was built with:
python version: 3.8.15 (default, Nov  4 2022, 15:16:59) [MSC v.1916 64 bit (AMD64)] 
PCSE version: 5.5.4


## 2.1 作物参数

In [6]:
from pcse.fileinput import YAMLCropDataProvider, CABOFileReader
from pcse.exceptions import PCSEError, PartitioningError

In [7]:
cropfile = os.path.join(data_dir, 'npkfile', 'wofost_npk.crop')
cropd = CABOFileReader(cropfile)


## 2.2 土壤参数

In [8]:
soilfile = os.path.join(data_dir, 'soil', 'ec3_copy1.soil')
soild = CABOFileReader(soilfile)

## 2.3 站点数据

In [9]:
from pcse.util import WOFOST80SiteDataProvider
sited = WOFOST80SiteDataProvider(WAV=18, NAVAILI=20.5, PAVAILI=50.0,KAVAILI=250.0)

## 2.4 整合模型参数

In [10]:
from pcse.base import ParameterProvider
# parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

In [11]:
# cropfile = os.path.join(data_dir, 'crop', 'MAG201.CAB')
# crop_param = CropParam(cropfile, soild, sited)
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

## 2.5 管理文件

In [12]:
# 管理文件
with open("./data/npkfile/wofost_npk2022.agro", 'r') as f:
    yaml_agro_2022 = f.read()
with open("./data/npkfile/wofost_npk2021.agro", 'r') as f:
    yaml_agro_2021 = f.read()

In [13]:
def my_agro(agro_yaml, n_amount):
    yaml_agro = copy.deepcopy(agro_yaml)
    n1 = n_amount * 0.6
    n2 = n_amount - n1
    yaml_agro = yaml_agro.replace("My_N1", str(n1))
    yaml_agro = yaml_agro.replace("My_N2", str(n2))
    
    return yaml.safe_load(yaml_agro)['AgroManagement']

## 2.6气象数据

In [14]:
from pcse.fileinput import ExcelWeatherDataProvider
weatherfile = os.path.join(data_dir, 'meteo', 'WOFOSTYL.xlsx')
wdp = ExcelWeatherDataProvider(weatherfile)
print(wdp)

Weather data provided by: ExcelWeatherDataProvider
--------Description---------
Weather data for:
Country: China
Station: Yangling, Shannxi
Description: Observed data from Station Yangling in China
Source: Meteorology and Air Quality Group, Wageningen University
Contact: Peter Uithol
----Site characteristics----
Elevation:  521.0
Latitude:  34.330
Longitude: 108.120
Data available for 2021-06-01 - 2022-10-31
Number of missing days: 182



In [15]:
param_name = ["TDWI", "LAIEM", "RGRLAI", "SLATB", "SPAN", "TBASE", "KDIFTB", "EFFTB", "AMAXTB", "TMPFTB", "TMNFTB", "CVL", "CVO", "CVR", "CVS", "Q10", "RML",
              "RMO", "RMR", "RMS", "RFSETB", "FRTB", "FLTB", "FOTB", "RDRRTB", "RDRSTB", "CFET", "DEPNR", "RDI", "RRI", "RDMCR"]

In [16]:
for each in param_name:
    try:
        # print("**************\n")

        test = parameters[each]

    except KeyError:
        print(each)


# 3 敏感性分析

## 3.1敏感性分析设置

In [17]:
from pcse.models import Wofost80_NWLP_FD_beta as Wofost80_NWLP_FD

### 3.1.1设置提取的值 

In [18]:
def create_dict(key_das):
    res_dict = dict()
    res_dict["0"] = list()
    for each in key_das:
        dict_key = str(each)
        res_dict[dict_key] = list()
    return res_dict
#
#
# def get_wofost_output(wofostModel, var_names, template_dict, pid, var_list=None):
#     wofostModel.run_till_terminate()
#     r = wofostModel.get_summary_output()
#     tmp_res = parase_model_ouput(r, var_name=var_names, pid=pid, idx=0)
#     template_dict["0"].append(tmp_res)
#     if var_list is None:
#         pass
#     else:
#         days, name_ = var_list
#         r = wofostModel.get_output()
#         for ii in days:
#             # print(ii)
#             tmp_res = None
#             tmp_res = parase_model_ouput(r, var_name=name_, idx=ii, pid=pid)
#             template_dict[str(ii)].append(tmp_res)
#
#     return template_dict
#


In [19]:
final_target = ["LAIMAX", "TAGP", "TWSO"]
time_target = ["LAI", "TAGP"]
das = [i for i in range(126)]
model_start_date = "2021-06-01"
sowing_date = "2021-06-11"
base_i = calDays(model_start_date, sowing_date)
for idx, value_ in enumerate(das):
    das[idx] = base_i + value_

col1 = ["LAIMAX", "TAGP", "TWSO", "id"]
col2 = ["LAI", "TAGP", "id"]

In [20]:
def my_crop_dict():

    mycropd = {

        "AMAXTB": [0.0, 70.0, 
                   1.25, 70.0, 
                   1.5, 63.0, 
                   1.75, 49.0, 
                   2.0, 21.0],

        "TMPFTB": [0.00, 0.01, 
                   9.00, 0.05, 
                   16.0, 0.8, 
                   18.0, 0.94, 
                   20.0, 1.0, 
                   30.0, 1.0, 
                   36.0, 0.95, 
                   42.0, 0.56],

        "TMNFTB": [5.0, 0.0, 8.0, 1.0],

        "RFSETB": [0.0, 1.0, 
                   1.5, 1.0, 
                   1.75, 0.75, 
                   2.0, 0.25],

        "SLATB": [0.0, 0.0026, 
                  0.78, 0.001, 
                  2.00, 0.0012],

        "KDIFTB": [0.0, 0.6, 2.0, 0.6],

        "EFFTB": [0.0, 0.45, 40.0, 0.45],

        "FLTB": [0.00, 0.62, 
                 0.33, 0.62, 
                 0.88, 0.15, 
                 0.95, 0.15, 
                 1.10, 0.10, 
                 1.20, 0.00, 
                 2.00, 0.00],

        "FOTB": [0.00, 0.00, 
                 0.33, 0.00, 
                 0.88, 0.00, 
                 0.95, 0.00, 
                 1.10, 0.50, 
                 1.20, 1.00, 
                 2.00, 1.00],

        "FSTB": [0.00, 0.38, 
                 0.33, 0.38, 
                 0.88, 0.85, 
                 0.95, 0.85, 
                 1.10, 0.40, 
                 1.20, 0.00, 
                 2.00, 0.00],
        "NMAXLV_TB": [0.0, 0.06,
                      0.4, 0.04,
                      0.7, 0.03,
                      1.0, 0.02,
                      2.0, 0.014,
                      2.1, 0.014,]
        }

    return mycropd

In [21]:
import yaml
import numpy as np
import pandas as pd
# from SALib.sample import saltelli
from SALib.analyze import fast
# import matplotlib.pyplot as plt
from IPython.display import display
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 250)
# from progressbar import printProgressBar, PrintProgressBar
import time
import pickle
# from progressbar import printProgressBar, PrintProgressBar
from wofostSA1 import run_wofost, convert2simlab1

## 选择参数集

In [22]:
with open(os.path.join(data_dir, 'yaml', 'params2.yaml'), 'r', encoding='utf-8') as f:
    problem_yaml = f.read()
problem = yaml.safe_load(problem_yaml)

## 读取数据样本

In [23]:
with open("./NLWP1.sam", "r", encoding="utf-8") as f:
    problem_name, bounds1 = list(), list()
    tmp_list, res_list = list(), list()
    count = 0
    f.seek(0)
    flag = 0
    flag2 = 0
    for num, line in enumerate(f):
        if num == 1:
            total = int(line)
            continue
        elif num == 2:
            num_var = int(line)
            continue

        elif num > 2:

            try:
                line = line.rstrip()
                tmp_list = line.split('\t')
                # print(len(tmp_list))
                if len(tmp_list) == num_var and count < total:
                    res_list.append(tmp_list)
                    count += 1
                else:
                    temp_str = f"{num_var} Distributions"
                    if temp_str in line:
                        start1 = num + 2
                        flag = 1
                        count1 = 0
                        count2 = 0
                    
                    if flag and count1 < num_var and num == start1 + count1*6:
                        start2 = num
                        problem_name.append(line)
                        count1 += 1
                        flag2 = 1

                    if flag2 and count2 < num_var and num == start2 + 3:
                        __tmp = line.split("\t")
                        print(__tmp)
                        bounds1.append([float(__tmp[0]), float(__tmp[1])])
                        count2 += 1

            except ValueError:
                pass


paramsets = np.array(res_list, dtype=np.float64)

['0.0013', '0.0039', '1']
['0.0006', '0.0018', '1']
['0.0006', '0.0018', '1']
['0.3', '0.9', '1']
['0.3', '0.9', '1']
['0.225', '0.675', '1']
['0.225', '0.675', '1']
['35', '105', '1']
['31.5', '94.5', '1']
['24.5', '73.5', '1']
['35', '105', '1']
['10.5', '31.5', '1']
['0.005', '0.015', '1']
['0.5', '1.5', '1']
['0.5', '1.5', '1']
['0.5', '1.5', '1']
['0.125', '0.375', '1']
['0.34', '1.02', '1']
['0.3355', '1.0065', '1']
['0.345', '1.035', '1']
['0.329', '0.987', '1']
['1', '3', '1']
['0.015', '0.045', '1']
['0.005', '0.015', '1']
['0.0075', '0.0225', '1']
['0.0075', '0.0225', '1']
['0.015', '0.045', '1']
['5', '15', '1']
['2', '6', '1']
['15', '45', '1']
['25', '75', '1']
['0.0147', '0.0441', '1']
['16.5', '49.5', '1']
['1.1', '3.3', '1']
['0.465', '0.775', '1']
['0.465', '0.775', '1']
['0.075', '0.225', '1']
['0.075', '0.225', '1']
['0.08', '0.12', '1']
['0.3', '0.7', '1']
['0.025', '0.075', '1']
['0.7', '2', '1']
['0.4', '1.2', '1']
['0.25', '0.75', '1']
['0.5', '1.5', '1']
['0.25'

In [24]:
paramsets.shape

(6405, 61)

In [25]:
problem_dict1 = dict()
for each in problem_name:
    for name, value in zip(problem["names"], problem["bounds"]):
        if each == name:
            problem_dict1[each] = value

In [26]:
bounds_list = list()

In [27]:
len(bounds1) == len(problem_name)

True

In [77]:
for each in problem_name:
    bounds_list.append(problem_dict1[each])

KeyError: 'NMAXLV_TB001'

In [28]:
problem = dict()
problem["bounds"] = bounds1
problem["names"] = problem_name
problem["num_vars"] = len(bounds1)

In [29]:
for name, bound in zip(problem["names"], problem["bounds"]):
    print("%s %s" % (name, bound))

SLATB001 [0.0013, 0.0039]
SLATB003 [0.0006, 0.0018]
SLATB005 [0.0006, 0.0018]
KDIFTB001 [0.3, 0.9]
KDIFTB003 [0.3, 0.9]
EFFTB001 [0.225, 0.675]
EFFTB003 [0.225, 0.675]
AMAXTB003 [35.0, 105.0]
AMAXTB005 [31.5, 94.5]
AMAXTB007 [24.5, 73.5]
AMAXTB001 [35.0, 105.0]
AMAXTB009 [10.5, 31.5]
TMPFTB001 [0.005, 0.015]
TMPFTB007 [0.5, 1.5]
TMNFTB003 [0.5, 1.5]
RFSETB001 [0.5, 1.5]
RFSETB007 [0.125, 0.375]
CVL [0.34, 1.02]
CVO [0.3355, 1.0065]
CVR [0.345, 1.035]
CVS [0.329, 0.987]
Q10 [1.0, 3.0]
RML [0.015, 0.045]
RMO [0.005, 0.015]
RMR [0.0075, 0.0225]
RMS [0.0075, 0.0225]
PERDL [0.015, 0.045]
RDI [5.0, 15.0]
TBASEM [2.0, 6.0]
TEFFMX [15.0, 45.0]
TDWI [25.0, 75.0]
RGRLAI [0.0147, 0.0441]
SPAN [16.5, 49.5]
RRI [1.1, 3.3]
FLTB001 [0.465, 0.775]
FLTB003 [0.465, 0.775]
FLTB005 [0.075, 0.225]
FLTB007 [0.075, 0.225]
FLTB009 [0.08, 0.12]
FOTB009 [0.3, 0.7]
RDRLV_NPK [0.025, 0.075]
DVS_NPK_STOP [0.7, 2.0]
DVS_NPK_TRANSL [0.4, 1.2]
NPK_TRANSLRT_FR [0.25, 0.75]
NCRIT_FR [0.5, 1.5]
NMAXRT_FR [0.25, 0.75]
NM

In [30]:
nsets = len(paramsets)
result_template = create_dict(das)
# result_wlp = create_dict(das)
result_template["names1"] = col1
result_template["names2"] = col2
# result_wlp["names1"] = col1
# result_wlp["names2"] = col2

In [31]:
def parase_res_dict(input_dict: dict(), prefix):
    """
    @param prefix: must contain "_s_"
    """
    tmp_df = None
    for key_ in input_dict.keys():
        
        if key_.isnumeric():
            file_name = prefix.replace("_s_", key_)
            if key_ == "0":
                tmp_df = pd.DataFrame(input_dict[key_], columns=input_dict["names1"])
            else:
                tmp_df = pd.DataFrame(input_dict[key_], columns=input_dict["names2"])

            convert2simlab1(tmp_df, file_name)


In [32]:
def gen_res_dict(input_dict: dict):
    tmp_df = None
    for key_ in input_dict.keys():
        
        if key_.isnumeric():
            if key_ == "0":
                name = input_dict["names1"]
                tmp_df = pd.DataFrame(input_dict[key_], columns=input_dict["names1"])
            else:
                name = input_dict["names2"]
                tmp_df = pd.DataFrame(input_dict[key_], columns=input_dict["names2"])
            
            yield (key_, name, tmp_df)

In [33]:
def save_pickle(data, pkl_path):
    with open(pkl_path, "wb") as f:
        pickle.dump(data, f)
        print(f"{pkl_path}保存成功\n")
     

### efast

In [34]:
save_dir = "./data/pickleFile/12161/"

In [35]:
save_dir = "./data/pickleFile/12161/"
n_amounts = [180]
for nn in n_amounts:
    parameters.clear_override()
    result_pp = copy.deepcopy(result_template)
    pklfile = "%sPPN%s2021.pkl" % (save_dir, nn)
    agromanagement21 = my_agro(yaml_agro_2021, nn)
    agromanagement22 = my_agro(yaml_agro_2022, nn)
    result_pp = run_wofost(variable_name=problem["names"], paramsets=paramsets, result_dict=result_pp, parameters=parameters,
                        agrodata=agromanagement21, weatherdata=wdp, pkl_file=pklfile, target_variables=final_target, target_list=(das, time_target), phenology=True)
    # parase_res_dict(result_2021pp, "./data/modelOut/2021PP/2021PP_s_.psc")
    save_pickle(result_pp, pklfile)
    pklfile = "%sPPN%s2022.pkl" % (save_dir, nn)
    result_pp = copy.deepcopy(result_template)
    result_pp = run_wofost(variable_name=problem["names"], paramsets=paramsets, result_dict=result_pp, parameters=parameters,
                            agrodata=agromanagement22, weatherdata=wdp, pkl_file=pklfile, target_variables=final_target, target_list=(das, time_target), phenology=True)
    save_pickle(result_pp, pklfile)
    # parase_res_dict(result_2022pp, "./data/modelOut/2022PP/2022PP_s_.psc")

完成进度: 100%|████████████████| 6405/6405 [48:34<00:00,  2.20it/s, number=6404]ing]


./data/pickleFile/12161/PPN1802021.pkl保存成功



完成进度: 100%|████████████████| 6405/6405 [46:42<00:00,  2.29it/s, number=6404]ing]


./data/pickleFile/12161/PPN1802022.pkl保存成功



**暂时不用**

In [14]:
with open("./fastsample33LAI.sam", "r", encoding="utf-8") as f:
    problem_name, bounds1 = list(), list()
    tmp_list, res_list = list(), list()
    count = 0
    f.seek(0)
    flag = 0
    flag2 = 0
    for num, line in enumerate(f):
        if num == 1:
            total = int(line)
            continue
        elif num == 2:
            num_var = int(line)
            continue

        elif num > 2:

            try:
                line = line.rstrip()
                tmp_list = line.split('\t')
                # print(len(tmp_list))
                if len(tmp_list) == num_var and count < total:
                    res_list.append(tmp_list)
                    count += 1
                else:
                    temp_str = f"{num_var} Distributions"
                    if temp_str in line:
                        start1 = num + 2
                        flag = 1
                        count1 = 0
                        count2 = 0

                    if flag and count1 < num_var and num == start1 + count1*6:
                        start2 = num
                        problem_name.append(line)
                        count1 += 1
                        flag2 = 1

                    if flag2 and count2 < num_var and num == start2 + 3:
                        __tmp = line.split("\t")
                        print(__tmp)
                        bounds1.append([float(__tmp[0]), float(__tmp[1])])
                        count2 += 1

            except ValueError:
                pass


paramsets = np.array(res_list, dtype=np.float64)
problem["bounds"] = bounds1
problem["names"] = problem_name
problem["num_vars"] = len(bounds1)

['0.0013', '0.0039', '1']
['0.0006', '0.0018', '1']
['0.0006', '0.0018', '1']
['0.3', '0.9', '1']
['0.3', '0.9', '1']
['0.225', '0.675', '1']
['0.225', '0.675', '1']
['35', '105', '1']
['31.5', '94.5', '1']
['24.5', '73.5', '1']
['35', '105', '1']
['10.5', '31.5', '1']
['0.005', '0.015', '1']
['0.5', '1.5', '1']
['0.5', '1.5', '1']
['0.5', '1.5', '1']
['0.125', '0.375', '1']
['0.34', '1.02', '1']
['0.3355', '1.0065', '1']
['0.345', '1.035', '1']
['0.329', '0.987', '1']
['1', '3', '1']
['0.015', '0.045', '1']
['0.005', '0.015', '1']
['0.0075', '0.0225', '1']
['0.0075', '0.0225', '1']
['0.015', '0.045', '1']
['5', '15', '1']
['2', '6', '1']
['15', '45', '1']
['25', '75', '1']
['0.0147', '0.0441', '1']
['16.5', '49.5', '1']
['1.1', '3.3', '1']
['0.465', '0.775', '1']
['0.465', '0.775', '1']
['0.075', '0.225', '1']
['0.075', '0.225', '1']
['0.08', '0.12', '1']
['0.3', '0.7', '1']


In [ ]:
save_dir = "./data/pickleFile/LAI42/"
n_amounts = [0, 90, 180, 270]
for nn in n_amounts:
    parameters.clear_override()
    result_pp = copy.deepcopy(result_template)
    pklfile = "%sPPN%s2021.pkl" % (save_dir, nn)
    agromanagement21 = my_agro(yaml_agro_2021, nn)
    agromanagement22 = my_agro(yaml_agro_2022, nn)
    result_pp = run_wofost(variable_name=problem["names"], paramsets=paramsets, result_dict=result_pp, parameters=parameters,
                           agrodata=agromanagement21, weatherdata=wdp, pkl_file=pklfile, target_variables=final_target, target_list=(das, time_target))
    # parase_res_dict(result_2021pp, "./data/modelOut/2021PP/2021PP_s_.psc")
    save_pickle(result_pp, pklfile)
    pklfile = "%sPPN%s2022.pkl" % (save_dir, nn)
    result_pp = copy.deepcopy(result_template)
    result_pp = run_wofost(variable_name=problem["names"], paramsets=paramsets, result_dict=result_pp, parameters=parameters,
                           agrodata=agromanagement22, weatherdata=wdp, pkl_file=pklfile, target_variables=final_target, target_list=(das, time_target))
    save_pickle(result_pp, pklfile)
    # parase_res_dict(result_2022pp, "./data/modelOut/2022PP/2022PP_s_.psc")

In [36]:
def isdir_demo(dir_path):
    if not os.path.isdir(dir_path):
        try:
            os.mkdir(dir_path)
        except FileNotFoundError:
            os.makedirs(dir_path)

In [37]:
import sqlite3
from tqdm import tqdm
import pickle
from SALib.analyze import fast

In [40]:
root_dir = "./data/modelOut/NWLP1/"
save_dir1 = "./data/modelOut/NWLP1/Siraw/"
isdir_demo(save_dir1)
connLAI = sqlite3.connect(os.path.join(save_dir1, "LAI_ori.db"))
for dir_root, _, file_list in os.walk(root_dir):
    for file_name in file_list:
        save_name = file_name.split(".")[0]
        year_ = save_name[-4: ]
        table_name = save_name[:-4]
        with open(os.path.join(dir_root, file_name), 'rb') as f:
            result_pp = pickle.load(f)
        
        gen_dict = gen_res_dict(result_pp)
        for num_, names_l, df1 in tqdm(gen_dict):
            for each in names_l[: -1]:
                Y = df1[each]
                Y = Y.to_numpy()
                Si = fast.analyze(problem, Y, print_to_console=False)
                df_sensitivity = pd.DataFrame({
                    "Parameter": problem["names"],
                    "First_order": Si["S1"],
                    "Total_order": Si["ST"]}).set_index("Parameter")
                if num_ == "0":
                    save_name1 = f"{save_name}_{each}_{num_}.csv"
                    df_sensitivity.to_csv(os.path.join(save_dir1, save_name1))
                else:
                    df_sensitivity['YEAR'] = int(year_)
                    df_sensitivity['DOY'] = int(num_)
                    df_sensitivity.to_sql(f"{table_name}{each}", con=connLAI, chunksize=10000, if_exists='append')

connLAI.close()


127it [01:53,  1.12it/s]
127it [01:54,  1.11it/s]


UnpicklingError: the STRING opcode argument must be quoted

In [42]:
connLAI.close()


[1, 2, 3, 4]

In [40]:
df_pp = pd.DataFrame(result_2021wlp["0"], columns=result_2021wlp["names1"])

In [ ]:
Y = df_pp["TWSO"]

In [ ]:
Y = Y.to_numpy()

In [ ]:
Y

In [ ]:
Si = fast.analyze(problem, Y, print_to_console=False)

In [ ]:
df_sensitivity = pd.DataFrame({
    "Parameter": problem["names"],
    "First_order": Si["S1"],
    "Total_order": Si["ST"]}).set_index("Parameter")
fig, axes = plt.subplots(figsize=(10,6))
df_sensitivity.plot(kind="bar", ax=axes)

In [ ]:
df_sensitivity